Imports

In [ ]:
!pip install scikeras==0.12.0
!pip install missingpy==0.2.0
!pip install statsmodels
!pip uninstall -y cvxpy
!pip install cvxpy==1.5.3

In [ ]:
import os
import random

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm

from scipy.stats import norm

from sklearn import ensemble
from sklearn.linear_model import LinearRegression


import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
#from keras.wrappers.scikit_learn import KerasClassifier
#from keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasClassifier, KerasRegressor
#import shap

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import pickle

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import sklearn.metrics

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.tree import export_text

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import collections

import time

import pickle


import random

from pandas.compat import numpy
import scipy


Data generation (dat.py)



In [ ]:

def gen_sim_data(n, s_type, y_set):
    """generate simulated data"""


    if (y_set == "toy"):

        y_fn = "(X1>0.5)*(24 - 11*A - 19*(1-cos(3.1415*0.5*S)) + 21*A*(1-cos(3.1415*0.5*S))) + (X1<=0.5)*(11 + 14*A + 2*(1-cos(3.1415*0.5*S)) - 27*A*(1-cos(3.1415*0.5*S)))"

        X1 = np.random.normal(0.5,1,n)
        X2 = np.random.uniform(0,0, n)
        err = np.random.normal(loc=0.0, scale=1, size=n)

        if s_type == "cont":
            s1 = np.random.beta(4,1,n//2)
            s0 = np.random.beta(1,4,n-n//2)
            S = np.concatenate([s0,s1])
            random.shuffle(S)

        elif s_type == "disc":
            S = np.random.binomial(1, 0.5, n)
        A = np.random.binomial(1, 0.5, n)
        dat = np.concatenate((A[:,np.newaxis], S[:,np.newaxis], X1[:,np.newaxis], X2[:,np.newaxis]),axis=1) #, X3[:,np.newaxis], X4[:,np.newaxis]
        df = pd.DataFrame(data=dat, columns=['A','S','X1','X2'])
        df["Y"] = df.eval(y_fn) + err



    if (y_set == "complex"):
        y_fn = "(0.5 + 1*A + exp(S) - 2.5*A*S) * (1+X1 -X2 +X3**2 +exp(X4)) + (1 + 2*A + 0.2*exp(S) - 3.5*A*S) * (1+5*X1 -2*X2 +3*X3 +2*exp(X4))"
        X1 = np.random.uniform(0,1, n)
        X2 = np.random.uniform(0,1, n)
        X3 = np.random.uniform(0,1, n)
        X4 = np.random.uniform(0,1, n)
        X5 = np.random.uniform(0,1, n)
        X6 = np.random.uniform(0,1, n)
        err = np.random.normal(loc=0.0, scale=1, size=n)

        pr_s1 = 1 / (1 + np.exp( 2.5 - 0.8*(X1+X2+X3+X4+X5+X6) )) # expit(-2.5 + 0.8()) # norm

        if s_type == "cont":
            s1 = np.random.beta(4,1,n//2)
            s0 = np.random.beta(1,4,n-n//2)
            S = np.concatenate([s0,s1])
        else:
            S = np.random.binomial(1, pr_s1, n)

        dat = np.concatenate((S[:,np.newaxis], X1[:,np.newaxis], X2[:,np.newaxis], X3[:,np.newaxis], X4[:,np.newaxis], X5[:,np.newaxis], X6[:,np.newaxis]),axis=1)
        df = pd.DataFrame(data=dat, columns=['S','X1','X2','X3','X4','X5','X6'])


        a_fn = "1 / (1 + exp( 0.6*S - 0.6*X1 + 0.6*X2 - 0.6*X3 + 0.6*X4 - 0.6*X5 + 0.6*X6 ))"
        prop_sc = df.eval(a_fn)
        df['A'] = np.random.binomial(1, prop_sc, n)

        df["Y"] = df.eval(y_fn) + err


    # convert to float
    cols=[i for i in df.columns]
    for col in cols:
        df[col]=pd.to_numeric(df[col])

    df["A"] = df["A"].astype(int)

    if (s_type == "disc"):
        df["S"] = df["S"].astype(int)

    return(df, y_fn)

In [ ]:

def gen_sim_data_RWDRCT(df,n):
    Spprob=np.exp(-5+3*df['X1']-2*df['S'])/(1+np.exp(-5+3*df['X1']-2*df['S']))
    Sp=np.random.binomial(1,Spprob,n)
    df["Spprob"]=Spprob
    df["Sp"]=Sp
    return df

N=100000


def transfer_weight_obj(alpha):
    m=df_RWD.shape[0]
    df_RCT_1X=np.concatenate((np.ones((df_RCT.shape[0],1)),df_RCT[["X1","X2","S"]]),axis=1)
    df_RWD_1X=np.concatenate((np.ones((df_RWD.shape[0],1)),df_RWD[["X1","X2","S"]]),axis=1)
    return -(1/N)*np.sum(df_RCT_1X@alpha)+(1/m)*np.sum(np.log(1+np.exp(df_RWD_1X@alpha)))





Randomization & Normalization & Split train test

In [ ]:

def set_random(seed_value):
    os.environ['PYTHONHASHSEED']=str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)

    return


def normalize(df_tr, df_te, names_to_norm):
    df = df_tr.copy()
    df_test = df_te.copy()

    # normalize X & S
    scaler = StandardScaler()
    df[names_to_norm] = scaler.fit_transform(df[names_to_norm])
    df_test[names_to_norm] = scaler.transform(df_test[names_to_norm])

    return(df.copy(), df_test.copy())


def get_train_test(df_all, s_type, seed_value):
    # train-test split by A
    df0_ori = df_all[df_all["A"]==0].reset_index(drop=True).copy()
    df1_ori = df_all[df_all["A"]==1].reset_index(drop=True).copy()
    df0_train, df0_test = train_test_split(df0_ori, test_size=0.2, random_state=seed_value)
    df1_train, df1_test = train_test_split(df1_ori, test_size=0.2, random_state=seed_value)

    df_ori = pd.concat([df0_train, df1_train]).reset_index(drop=True).copy()
    df_test_ori = pd.concat([df0_test, df1_test]).reset_index(drop=True).copy()


    print("df_ori.shape", df_ori.shape)
    if (s_type == "disc"):
        print("train Pr(S=0)/Pr(S=1):", round(sum(df_ori["S"]==0) / sum(df_ori["S"]==1),3))
    print("train Pr(A=0)/Pr(A=1):", round(sum(df_ori["A"]==0) / sum(df_ori["A"]==1),3))

    print("df_test_ori.shape", df_test_ori.shape)
    if (s_type == "disc"):
        print("test Pr(S=0)/Pr(S=1):", round(sum(df_test_ori["S"]==0) / sum(df_test_ori["S"]==1),3))
    print("test Pr(A=0)/Pr(A=1):", round(sum(df_test_ori["A"]==0) / sum(df_test_ori["A"]==1),3))

    return(df_ori.copy(), df_test_ori.copy())



Get PS fit & Keras & Hyper tuning

In [ ]:

def get_ps_fit(df_train, use_covars, transfer_weights):
    df_tr = df_train.copy()
    regr = RandomForestClassifier(random_state=1234).fit(df_tr[use_covars], df_tr["A"],sample_weight=transfer_weights)
    return(regr)

def keras_wrap(x_train, train_labels, train_wts, x_test, loss_fn, act_out,
               layer=2, node=1024, dropout=0.2, n_epoch=100, bsize=64, act="relu",
               opt="Adam", val_split=0.2, is_early_stop=True, verb=0):

    if is_early_stop:
        early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
        callback = [early_stop]
    else:
        callback = None

    # set input_dim for the number of features
    if len(x_train.shape) == 1:
        input_dim = 1
    else:
        input_dim = x_train.shape[1]

    # model
    model = Sequential()
    for i in range(layer):
        if i==0:
            model.add(Dense(node, input_dim=input_dim, activation=act)) # Hidden 1
            model.add(Dropout(dropout))
        else:
           model.add(Dense(node, activation=act)) # Hidden 2
           model.add(Dropout(dropout))

    model.add(Dense(1, activation=act_out)) # Output

    model.compile(loss=loss_fn, optimizer=opt, weighted_metrics='acc')
    model.fit(x_train, train_labels,
              sample_weight=train_wts,
              epochs=n_epoch, batch_size=bsize,
              validation_split=val_split, callbacks=callback, verbose=verb)

    # predict
    pred_test = model.predict(x_test).flatten()
    pred_train = model.predict(x_train).flatten()
    return pred_test, pred_train, model


def hyper_tuning(x_train, train_labels, train_wts, loss_fn, act_out,
                 param_grid, n_cv=5, n_jobs=1):
    """
    layers = [2,3]
    nodes=[100,300,512]
    dropout=[0.2] #,0.4
    activation = ["sigmoid","relu"]
    optimizer = ["adam"] #,"nadam"
    bsize = [32,64] #,128
    n_epochs = [50,100] #,200
    bst_params = hyper_tuning(x_train, train_labels, train_wts, loss_fn, act_out,
                              param_grid, n_cv=5, n_jobs=1)
    """
    # set input_dim for the number of features
    if len(x_train.shape) == 1:
        input_dim = 1
    else:
        input_dim = x_train.shape[1]

    def create_model(layers,nodes,activation,optimizer,dropout):
        model = Sequential()
        for i in range(layers):
            if i==0:
                model.add(Dense(nodes, input_dim=input_dim))
                model.add(Activation(activation))
                model.add(Dropout(dropout))
            else:
                model.add(Dense(nodes, activation=activation))
                model.add(Activation(activation))
                model.add(Dropout(dropout))

        model.add(Dense(units=1, activation=act_out))

        model.compile(optimizer=optimizer, loss=loss_fn, metrics=['acc'],weighted_metrics=['acc'])
        return model

    if act_out == "sigmoid": #for classification
        model = KerasClassifier(build_fn=create_model, verbose=0)
    else: #None #for regression including quantile
        model = KerasRegressor(build_fn=create_model, verbose=0)

    assert param_grid is not None
    layers = param_grid['layers']
    nodes = param_grid['nodes']
    dropout = param_grid['dropouts']
    activation = param_grid['acts']
    optimizer = param_grid['opts']
    bsizes = param_grid['bsizes']
    n_epochs = param_grid['n_epochs']

    param_grid = dict(layers=layers, nodes=nodes, activation=activation, optimizer=optimizer,
                      dropout=dropout, batch_size=bsizes, epochs=n_epochs)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=n_cv, n_jobs=n_jobs)

    grid_result = grid.fit(x_train, train_labels, sample_weight=train_wts)

    print("hyperparams tuning:", grid_result.best_score_,grid_result.best_params_)

    bst_params = grid_result.best_params_
    layer = bst_params['layers']
    node = bst_params['nodes']
    dropout = bst_params['dropout']
    n_epoch = bst_params['epochs']
    bsize = bst_params['batch_size']
    act = bst_params['activation']
    opt = bst_params['optimizer']

    return layer, node, dropout, n_epoch, bsize, act, opt


Fit expectation & Fit quantiles

In [ ]:



def fit_expectation(obj_name, df, use_covars, df_pred, is_class, is_tune, param_grid, transfer_weights, df_val=None):
    """E(Y|X=x,A=a) or E(Y|X=x,S=s,A=a)
    (model separated by A)
    """

    df_tr = df.copy()
    df_te = df_pred.copy()
    if df_val is not None:
        df_va = df_val.copy()

    # fit on df0/df1
    if is_class:
        loss_fn = 'binary_crossentropy'
        act_out = 'sigmoid'
    else:
        loss_fn = "mean_squared_error"
        act_out = None

    if is_tune:
        layer, node, dropout, n_epoch, bsize, act, opt = \
                hyper_tuning(df_tr[use_covars], df_tr["Y"], transfer_weights, loss_fn, act_out,
                    param_grid, n_cv=5, n_jobs=1)
        Yhat, _, regr = keras_wrap(df_tr[use_covars], df_tr["Y"], transfer_weights,
                            df_te[use_covars], loss_fn, act_out,
                            layer, node, dropout, n_epoch, bsize, act, opt,
                            val_split=None, is_early_stop=False, verb=0)
    else:
        Yhat, _, regr = keras_wrap(df_tr[use_covars], df_tr["Y"], transfer_weights,
                            df_te[use_covars], loss_fn, act_out,
                            layer=2, node=1024, dropout=0.2, n_epoch=100, bsize=64, act="relu", opt="Adam",
                            val_split=0.2, is_early_stop=True, verb=0)

    if df_val is not None:
        if not is_class: #NRMSE
            y_tr = regr.predict(df_tr[use_covars])
            rms = mean_squared_error(df_tr['Y'], y_tr, squared=False)
            met = rms / (np.max(df_tr['Y']) - np.min(df_tr['Y']))
            print(obj_name, "evaluate on train: NRMSE", met)

            y_va = regr.predict(df_va[use_covars])
            rms = mean_squared_error(df_va['Y'], y_va, squared=False)
            met = rms / (np.max(df_va['Y']) - np.min(df_va['Y']))
            print(obj_name, "evaluate on test : NRMSE", met)
        elif is_class: #AUC
            y_tr = regr.predict(df_tr[use_covars])
            met = roc_auc_score(df_tr['Y'], y_tr)
            print(obj_name, "evaluate on train: AUC", met)

            y_va = regr.predict(df_va[use_covars])
            met = roc_auc_score(df_va['Y'], y_va)
            print(obj_name, "evaluate on test : AUC", met)

    return(Yhat, regr)


def fit_quantile(x_train, train_labels, x_test, q, is_tune, param_grid, transfer_weights):
    """
    quantile regression: yhat | X,A
    """

    def tilted_loss(q, y, f):
        """quantile loss for Keras"""

        e = (y - f)
        return keras.backend.mean(keras.backend.maximum(q * e, (q - 1) * e), axis=-1)

    loss_fn = lambda y, f: tilted_loss(q, y, f)
    act_out = None

    if is_tune:
        layer, node, dropout, n_epoch, bsize, act, opt = \
                    hyper_tuning(x_train, train_labels, transfer_weights, loss_fn, act_out,
                        param_grid, n_cv=5, n_jobs=1)
        pred_test, _, model = keras_wrap(x_train, train_labels, transfer_weights,
                        x_test, loss_fn, act_out,
                        layer, node, dropout, n_epoch, bsize, act, opt,
                        val_split=None, is_early_stop=False, verb=0)
    else:
        pred_test, _, model = keras_wrap(x_train, train_labels, transfer_weights, x_test, loss_fn, act_out,
                layer=2, node=1024, dropout=0.2, n_epoch=100, bsize=64, act="relu",
                opt="Adam", val_split=0.2, is_early_stop=True, verb=0)

    return model, pred_test



def fit_infinite(df, X_names, XS_names, fit):
    df_use = df.copy()

    S_names = list( set(XS_names) - set(X_names) )

    if len(S_names) == 1:
        s_min = int(np.min(df_use["S"]))
        s_max = int(np.max(df_use["S"]))
        s_grid = range(s_min, s_max+1)
    else:
        s_lst = []
        for s_var in S_names:
            s_min = int(np.min(df_use[s_var]))
            s_max = int(np.max(df_use[s_var]))
            s_grid = range(s_min, s_max+1)
            s_lst.append(np.array(s_grid))
        s_grid = s_lst.copy()
        print("s_grid:", s_grid)

    X_df = df_use[X_names].copy()
    idx = range(X_df.shape[0])
    X_df["idx"] = idx

    if len(S_names) == 1:
        idx_s = np.array(np.meshgrid(idx, s_grid)).reshape(2, len(idx)*len(s_grid)).T
        idx_s = pd.DataFrame(idx_s, columns = ['idx','S'])
    else:
        idx_s_lst = [np.array(idx)] + s_grid
        idx_s = np.array(np.meshgrid(*np.array(idx_s_lst, dtype=object))).reshape(1+len(S_names), -1).T
        idx_s = pd.DataFrame(idx_s, columns = ['idx']+S_names)

    aug_df = X_df.merge(idx_s, on='idx', how='left').copy()

    # predict Y
    aug_df["pred"] = fit.predict(aug_df[XS_names])#????

    smry = aug_df.groupby(by="idx", as_index=False).agg({"pred":["min"]})

    return(smry.loc[:,("pred","min")].values.copy())




Get label & weight for classification

Binary classification with sample weights for decision rule by Keras

In [ ]:

def get_label_wt(g1, g0):
    """get label & weight for classification"""
    c1 = g1
    c2 = g0
    label = np.sign(c1 - c2)
    label = np.where(label==1, 1, 0)

    weight = np.abs(c1 - c2)


    return(label, weight)




def class_pred(df_train, df_test, use_covars, label, weight, s_type, is_tune, param_grid,transfer_weights):
    """binary classification with sample weights for decision rule by Keras
    """

    df_tr = df_train.copy()
    df_te = df_test.copy()

    loss_fn = 'binary_crossentropy'
    act_out = 'sigmoid'

    # classification (train & pred)
    if is_tune:
        layer, node, dropout, n_epoch, bsize, act, opt = \
                    hyper_tuning(df_tr[use_covars], label, weight*transfer_weights, loss_fn, act_out,
                        param_grid, n_cv=5, n_jobs=1)
        prob_test, prob_train, model = keras_wrap(df_tr[use_covars], label, weight*transfer_weights,
                        df_te[use_covars], loss_fn, act_out,
                        layer, node, dropout, n_epoch, bsize, act, opt,
                        val_split=None, is_early_stop=False, verb=0)
    else:
        prob_test, prob_train, model = keras_wrap(df_tr[use_covars], label, weight*transfer_weights,
                        df_te[use_covars], loss_fn, act_out,
                        layer=2, node=1024, dropout=0.2, n_epoch=100, bsize=64, act="relu",
                        opt="Adam", val_split=0.2, is_early_stop=True, verb=0)


    pred_test = np.where(prob_test>0.5, "yes", "no")
    pred_train = np.where(prob_train>0.5, "yes", "no")

    # print table
    print("pred_train table:", collections.Counter(pred_train.ravel()))
    print("pred_test table:", collections.Counter(pred_test.ravel()))
    if s_type == "disc": #disc S
        df_tr["pred_train"] = pred_train
        df_te["pred_test"] = pred_test
        print(pd.crosstab(df_tr["S"], df_tr["pred_train"], normalize='index'))
        print(pd.crosstab(df_te["S"], df_te["pred_test"], normalize='index'))
    print("="*30)

    return(pred_test, pred_train, model)



get_A_names & add A_pred to df



In [ ]:

def get_A_names(is_base, is_exp, is_expOracle, is_qua, is_inf):
    A_names = list()
    A_names.append("A_obs")
    if is_base:
        A_names.append("A_base")
    if is_exp:
        A_names.append("A_exp")
    if is_expOracle:
        A_names.append("A_expOracle")
    if is_qua or is_inf:
        A_names.append("A_qua")
    print("A_names:",A_names)

    return A_names


def add_Apred_Midx(df, A_names, A_base, A_exp, A_expOracle, A_qua):
    df_use = df.copy()

    # add A_pred to df
    df_use["A_obs"] = np.where(df_use["A"]==1, "yes","no")
    if "A_base" in A_names:
        df_use["A_base"] = A_base
    if "A_exp" in A_names:
        df_use["A_exp"] = A_exp
    if "A_expOracle" in A_names:
        df_use["A_expOracle"] = A_expOracle
    if "A_qua" in A_names:
        df_use["A_qua"] = A_qua

    return df_use.copy()


In [ ]:

def get_A_names(is_base, is_exp, is_MI, is_expOracle, is_qua, is_inf):
    A_names = list()
    A_names.append("A_obs")
    if is_base:
        A_names.append("A_base")
    if is_exp:
        A_names.append("A_exp")
    if is_MI:
        A_names.append("A_MI")
    if is_expOracle:
        A_names.append("A_expOracle")
    if is_qua or is_inf:
        A_names.append("A_qua")
    print("A_names:",A_names)

    return A_names


def add_Apred_Midx(df, A_names, A_base, A_exp, A_MI, A_expOracle, A_qua):
    df_use = df.copy()

    # add A_pred to df
    df_use["A_obs"] = np.where(df_use["A"]==1, "yes","no")
    if "A_base" in A_names:
        df_use["A_base"] = A_base
    if "A_exp" in A_names:
        df_use["A_exp"] = A_exp
    if "A_MI" in A_names:
        df_use["A_MI"] = A_MI
    if "A_expOracle" in A_names:
        df_use["A_expOracle"] = A_expOracle
    if "A_qua" in A_names:
        df_use["A_qua"] = A_qua

    return df_use.copy()


In [ ]:
def optimal_AS_inf(df, proj, is_sim, s_type, X_names, XS_names, y_fn, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1, df_pop):
    """
    Used for real data with disc S | simulation with disc/cont S

    get df with minority index
        input:  df_train for E(Y|X,S,A) and df_test for Yhat -> optimal A and S
        output: df_test with optimal A and S
    """
    df_use = df.copy()
    df_sgrid = df_pop.copy()

    # S grid from empirical test data
    a_grid = range(0, 2)
    if s_type == "disc":
        s_grid = range(0, 2)
        assert len(s_grid) == 2
    elif s_type == "cont":

        s_grid = df_sgrid["S"].sort_values().values.copy()

        assert len(s_grid) > 2
    else:
        assert(1 == 0)
    print("len(s_grid):", len(s_grid))

    # augment X with A&S grid
    a_s_grid = np.array(np.meshgrid(a_grid, s_grid)).reshape(2, len(a_grid)*len(s_grid)).T
    a_s_grid = pd.DataFrame(a_s_grid, columns = ['A','S'])
    idx_as = range(a_s_grid.shape[0])
    a_s_grid["idx_as"] = idx_as

    X_df = df_use[X_names].copy()
    idx_i = range(X_df.shape[0])
    X_df["idx_i"] = idx_i

    idxs = np.array(np.meshgrid(idx_as, idx_i)).reshape(2, len(idx_as)*len(idx_i)).T
    idxs = pd.DataFrame(idxs, columns = ['idx_as','idx_i'])

    aug_df = X_df.merge(idxs, on='idx_i', how='left').copy()
    aug_df = aug_df.merge(a_s_grid, on='idx_as', how='left').copy()

    # aug_df: each X with all combinations of (S,A)
    aug_df = aug_df[['idx_i','A','S']+X_names].copy()
    assert(aug_df.shape[0] == len(a_grid)*len(s_grid)*df_use.shape[0])
    aug_df_a0 = aug_df[aug_df["A"]==0].reset_index(drop=True).copy()
    aug_df_a1 = aug_df[aug_df["A"]==1].reset_index(drop=True).copy()

    # get E[Y|X,S,A=d(x)]
    if is_sim: # from true distn (simulation)
        aug_df_a0["EY"] = aug_df_a0.eval(y_fn)
        aug_df_a1["EY"] = aug_df_a1.eval(y_fn)
    else: # from fitted model (real data)
        aug_df_a0["EY"] = fit_Y_XS_A0.predict(aug_df_a0[XS_names])
        aug_df_a1["EY"] = fit_Y_XS_A1.predict(aug_df_a1[XS_names])

    # find quantile/inf wrt S
    def quan(x):
        return x.quantile(qua_use)

    if s_type == "disc":
        agg_fn1 = {"EY":["min"]}
        agg_fn2 = {"EY_min":"max"}
        agg_fn3 = {"EY_min":"min"}
    else:
        agg_fn1 = {"EY":[quan]}
        agg_fn2 = {"EY_quan":"max"}
        agg_fn3 = {"EY_quan":"min"}

    # min E(Y|X,S,A) separate for A=0 and A=1
    min_a0 = aug_df_a0.groupby(by="idx_i", as_index=False).agg(agg_fn1)
    min_a0.columns = ["idx_i"] + ['_'.join(col) for col in min_a0.columns[1:]]
    aug_df_a0_min = aug_df_a0.merge(min_a0, on='idx_i').copy()
    min_a1 = aug_df_a1.groupby(by="idx_i", as_index=False).agg(agg_fn1)
    min_a1.columns = ["idx_i"] + ['_'.join(col) for col in min_a1.columns[1:]]
    aug_df_a1_min = aug_df_a1.merge(min_a1, on='idx_i').copy()
    aug_df_cat = pd.concat([aug_df_a0_min, aug_df_a1_min]).reset_index(drop=True).copy()

    # find S*: min-min E(Y|X,S,A)
    min_df = aug_df_cat.groupby(by="idx_i", as_index=False).agg(agg_fn3).copy()
    min_min_df = aug_df_cat.merge(min_df, on='idx_i',suffixes=('', '_min'))

    # get S_opt (S*) vulnerable groups
    if s_type == "disc":
        msk1 = min_min_df['EY_min'] == min_min_df['EY_min_min']
        msk2 = min_min_df['EY'] <= min_min_df['EY_min_min']
    elif s_type == "cont":
        msk1 = min_min_df['EY_quan'] == min_min_df['EY_quan_min']
        msk2 = min_min_df['EY'] <= min_min_df['EY_quan_min']
    df_filter = min_min_df[msk1 & msk2].reset_index(drop=True).copy() # may contains more rows than df_use
    assert( df_filter.shape[0] >= df_use.shape[0] )

    X_df = df_use[X_names].copy()
    X_df["idx_i"] = idx_i
    df_S_opt = X_df.merge(df_filter, on=['idx_i']+X_names, how="left").copy()
    df_S_opt.rename(columns={"S":"S_opt"}, inplace=True)
    # df_S_opt should be used to plot tree (find pattern of X->S)

    # link S* to observed data
    XS_df = df_use[X_names+['S']].copy()
    XS_df["idx_i"] = idx_i
    merge_df = XS_df.merge(df_S_opt, left_on=['idx_i','S']+X_names, right_on=['idx_i','S_opt']+X_names, how="left").copy()
    merge_df = merge_df.drop_duplicates(subset=['idx_i']+X_names, keep='first').reset_index(drop=True).copy()

    S_opt = merge_df['S_opt'].copy() # if not NaN, this subject is vulnerable
    assert( S_opt.shape[0] == X_df.shape[0] )

    # minor_index
    merge_df['M_opt'] = np.where(merge_df['S']==merge_df['S_opt'], 1, 0)
    print("M_opt 1:minor 0:rest\n", merge_df['M_opt'].value_counts(dropna=False))
    M_opt = merge_df['M_opt'].copy()
    assert( M_opt.shape[0] == X_df.shape[0] )

    return S_opt, M_opt, df_S_opt # valid_df should be used to plot tree (find pattern of X->S)


In [ ]:
def optimal_AS_inf_train(df, proj, is_sim, s_type, X_names, XS_names, y_fn, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1, df_pop):
    """
    Used for real data with disc S | simulation with disc/cont S

    get df with minority index
        input:  df_train for E(Y|X,S,A) and df_test for Yhat -> optimal A and S
        output: df_test with optimal A and S
    """
    df_use = df.copy()
    df_sgrid = df_pop.copy()


    # S grid from empirical test data
    a_grid = range(0, 2)
    if s_type == "disc":
        s_grid = range(0, 2)
        assert len(s_grid) == 2
    elif s_type == "cont":

        s_grid = df_sgrid["S"].sort_values().values.copy()

        assert len(s_grid) > 2
    else:
        assert(1 == 0)
    print("len(s_grid):", len(s_grid))

    # augment X with A&S grid
    a_s_grid = np.array(np.meshgrid(a_grid, s_grid)).reshape(2, len(a_grid)*len(s_grid)).T
    a_s_grid = pd.DataFrame(a_s_grid, columns = ['A','S'])
    idx_as = range(a_s_grid.shape[0])
    a_s_grid["idx_as"] = idx_as

    X_df = df_use[X_names].copy()
    idx_i = range(X_df.shape[0])
    X_df["idx_i"] = idx_i

    idxs = np.array(np.meshgrid(idx_as, idx_i)).reshape(2, len(idx_as)*len(idx_i)).T
    idxs = pd.DataFrame(idxs, columns = ['idx_as','idx_i'])

    aug_df = X_df.merge(idxs, on='idx_i', how='left').copy()
    aug_df = aug_df.merge(a_s_grid, on='idx_as', how='left').copy()

    # aug_df: each X with all combinations of (S,A)
    aug_df = aug_df[['idx_i','A','S']+X_names].copy()
    assert(aug_df.shape[0] == len(a_grid)*len(s_grid)*df_use.shape[0])
    aug_df_a0 = aug_df[aug_df["A"]==0].reset_index(drop=True).copy()
    aug_df_a1 = aug_df[aug_df["A"]==1].reset_index(drop=True).copy()

    # get E[Y|X,S,A=d(x)]
    if is_sim: # from true distn (simulation)
        aug_df_a0["EY"] = aug_df_a0.eval(y_fn)
        aug_df_a1["EY"] = aug_df_a1.eval(y_fn)
    else: # from fitted model (real data)
        aug_df_a0["EY"] = fit_Y_XS_A0.predict(aug_df_a0[XS_names])
        aug_df_a1["EY"] = fit_Y_XS_A1.predict(aug_df_a1[XS_names])

    # find quantile/inf wrt S
    def quan(x,A):
        return x.quantile(qua_use)

    if s_type == "disc":
        agg_fn1 = {"EY":["min"]}
        agg_fn2 = {"EY_min":"max"}
        agg_fn3 = {"EY_min":"min"}
    else:
        agg_fn1 = {"EY":[quan]}
        agg_fn2 = {"EY_quan":"max"}
        agg_fn3 = {"EY_quan":"min"}

    # min E(Y|X,S,A) separate for A=0 and A=1
    min_a0 = aug_df_a0.groupby(by="idx_i", as_index=False).agg(agg_fn1)
    min_a0.columns = ["idx_i"] + ['_'.join(col) for col in min_a0.columns[1:]]
    aug_df_a0_min = aug_df_a0.merge(min_a0, on='idx_i').copy()
    min_a1 = aug_df_a1.groupby(by="idx_i", as_index=False).agg(agg_fn1)
    min_a1.columns = ["idx_i"] + ['_'.join(col) for col in min_a1.columns[1:]]
    aug_df_a1_min = aug_df_a1.merge(min_a1, on='idx_i').copy()
    aug_df_cat = pd.concat([aug_df_a0_min, aug_df_a1_min]).reset_index(drop=True).copy()

    # find S*: min-min E(Y|X,S,A)
    min_df = aug_df_cat.groupby(by="idx_i", as_index=False).agg(agg_fn3).copy()
    min_min_df = aug_df_cat.merge(min_df, on='idx_i',suffixes=('', '_min'))

    # get S_opt (S*) vulnerable groups
    if s_type == "disc":
        msk1 = min_min_df['EY_min'] == min_min_df['EY_min_min']
        msk2 = min_min_df['EY'] <= min_min_df['EY_min_min']
    elif s_type == "cont":
        msk1 = min_min_df['EY_quan'] == min_min_df['EY_quan_min']
        msk2 = min_min_df['EY'] <= min_min_df['EY_quan_min']
    df_filter = min_min_df[msk1 & msk2].reset_index(drop=True).copy() # may contains more rows than df_use
    assert( df_filter.shape[0] >= df_use.shape[0] )

    X_df = df_use[X_names].copy()
    X_df["idx_i"] = idx_i
    df_S_opt = X_df.merge(df_filter, on=['idx_i']+X_names, how="left").copy()
    df_S_opt.rename(columns={"S":"S_opt"}, inplace=True)
    # df_S_opt should be used to plot tree (find pattern of X->S)

    # link S* to observed data
    XS_df = df_use[X_names+['S']].copy()
    XS_df["idx_i"] = idx_i
    merge_df = XS_df.merge(df_S_opt, left_on=['idx_i','S']+X_names, right_on=['idx_i','S_opt']+X_names, how="left").copy()
    merge_df = merge_df.drop_duplicates(subset=['idx_i']+X_names, keep='first').reset_index(drop=True).copy()

    S_opt = merge_df['S_opt'].copy() # if not NaN, this subject is vulnerable
    assert( S_opt.shape[0] == X_df.shape[0] )

    # minor_index
    merge_df['M_opt'] = np.where(merge_df['S']==merge_df['S_opt'], 1, 0)
    print("M_opt 1:minor 0:rest\n", merge_df['M_opt'].value_counts(dropna=False))
    M_opt = merge_df['M_opt'].copy()
    assert( M_opt.shape[0] == X_df.shape[0] )

    return S_opt, M_opt, df_S_opt # valid_df should be used to plot tree (find pattern of X->S)


rule of vulnerable group (X -> S)

In [ ]:

def minor_rule(df, use_covars, s_type, S_name):
    """use decision tree to print out rule of vulnerable group (X -> S)
        only apply when S is discrete
    """
    df_use = df.copy()

    df_use["S_minor"] = np.where(np.isnan(df_use[S_name]), -999,df_use[S_name])
    print('df_use["S_minor"]', df_use["S_minor"].value_counts(dropna=False))

    if s_type == "disc":
        clf = DecisionTreeClassifier(max_depth=3, random_state=4211)
    else:
        clf = DecisionTreeRegressor(max_depth=3, random_state=4211)

    clf.fit(df_use[use_covars], df_use["S_minor"])

    # export the decision rules
    tree_rules = export_text(clf, feature_names = list(use_covars))
    print(S_name)
    print(tree_rules)

    clf.predict(df_use[use_covars])

    return


Expect reward & pred Y & value function for minor

In [ ]:


def expect_reward(df, A_name, minor_name):
    df_use = df.copy()

    minor_df = df_use[df_use[minor_name]==1].reset_index(drop=True).copy()
    rest_df = df_use[df_use[minor_name]==0].reset_index(drop=True).copy()

    numer = df_use["Y"] * (df_use["A_obs"] == df_use[A_name]) / df_use["ps"]
    denom = (df_use["A_obs"] == df_use[A_name]) / df_use["ps"]
    reward_all = np.sum(numer) / np.sum(denom)

    if not minor_df.empty:
        numer = minor_df["Y"] * (minor_df["A_obs"] == minor_df[A_name]) / minor_df["ps"]
        denom = (minor_df["A_obs"] == minor_df[A_name]) / minor_df["ps"]
        reward_minor = np.sum(numer) / np.sum(denom)
    else:
        reward_minor = np.nan

    if not rest_df.empty:
        numer = rest_df["Y"] * (rest_df["A_obs"] == rest_df[A_name]) / rest_df["ps"]
        denom = (rest_df["A_obs"] == rest_df[A_name]) / rest_df["ps"]
        reward_rest = np.sum(numer) / np.sum(denom)
    else:
        reward_rest = np.nan

    print(A_name, "reward  ",
          "all:",round(reward_all,3),
          "minor:",round(reward_minor,3),
          "rest:",round(reward_rest,3))

    return reward_all, reward_minor, reward_rest


def get_pred_Y(df, A_name, XS_names, y_fn, model_sxa0, model_sxa1):
    """generate Y based on predicted A (treatment assignment)
    model: Y|X,S,A=a
    should use unnormalized X & S for simulation & normalized for real data!
    """
    df_use = df[XS_names].copy()

    df_use["A"] = np.where(df[A_name]=="yes", 1,0)
    if y_fn is not None:
        Y_pred = df_use.eval(y_fn)
    else:
        Y_pred0 = model_sxa0.predict(df_use[XS_names]).flatten()
        Y_pred1 = model_sxa1.predict(df_use[XS_names]).flatten()
        Y_pred = np.where(df_use["A"]==0, Y_pred0, Y_pred1)
    assert len(Y_pred.shape)==1

    return Y_pred


def get_value_fn(df, A_name, XS_names, minor_name, y_fn, model_sxa0, model_sxa1):
    """get value function for subgroup of interest"""
    df_use = df.copy()

    df_use["Y_pred"] = get_pred_Y(df_use, A_name, XS_names, y_fn, model_sxa0, model_sxa1)

    minor_df = df_use[df_use[minor_name]==1].reset_index(drop=True).copy()
    rest_df = df_use[df_use[minor_name]==0].reset_index(drop=True).copy()

    value_all = np.mean(df_use["Y_pred"].values)

    if not minor_df.empty:
        value_minor = np.mean(minor_df["Y_pred"].values)
        minorPct=minor_df.shape[0]/df_use.shape[0]
    else:
        value_minor = np.nan
        minorPct=np.nan

    if not rest_df.empty:
        value_rest = np.mean(rest_df["Y_pred"].values)
    else:
        value_rest = np.nan

    print(A_name, "value   ",
          "all:",round(value_all,3),
          "minor:",round(value_minor,3),
          "rest:",round(value_rest,3),
          "minorPct:",round(minorPct,3))

    return value_all, value_minor, value_rest, minorPct

Objective Quantile & Inf

In [ ]:

def get_obj_qua(df, X_names, s_type, A_name, minor_name, fit_qua0, fit_qua1):
    """
    Used for real data with cont S (no need S grid)

    df includes X,A_test (A_test is from prediction)
    objective: E[Gs{E[Y|X,S,A=d(x)]}]
    -> Quantile_regress Y|X,A -> E[.]
    -> take avg over all subjects
    """
    df_use = df.copy()
    del df_use['A']
    del df_use['Y']

    if s_type != "cont":
        assert(1 == 0)

    Y_pred0 = fit_qua0.predict(df_use[X_names]).flatten()
    Y_pred1 = fit_qua1.predict(df_use[X_names]).flatten()

    df_use["A"] = np.where(df_use[A_name]=="yes", 1,0) #contains X & A
    df_use["Y"] = np.where(df_use["A"]==0, Y_pred0, Y_pred1)

    minor_df = df_use[df_use[minor_name]==1].reset_index(drop=True).copy()
    rest_df = df_use[df_use[minor_name]==0].reset_index(drop=True).copy()

    obj_all = np.mean(df_use["Y"].values)

    if not minor_df.empty:
        obj_minor = np.mean(minor_df["Y"].values)
    else:
        obj_minor = np.nan

    if not rest_df.empty:
        obj_rest = np.mean(rest_df["Y"].values)
    else:
        obj_rest = np.nan

    print(A_name, "obj     ",
          "all:",round(obj_all,3),
          "minor:",round(obj_minor,3),
          "rest:",round(obj_rest,3))

    return obj_all, obj_minor, obj_rest


def get_obj_inf(df, X_names, XS_names, A_name, minor_name, y_fn, s_type, is_class, qua_use, model_sxa0, model_sxa1):
    """
    Used for real data with disc S | simulation with disc/cont S
    df includes X,A_test (A_test is from prediction)
    objective: E[Gs{E[Y|X,S,A=d(x)]}]
    -> get E[Y|X,S,A=d(x)] from true distn (simulation) or fitted model (real data)
    -> given X & A, generate S from empirical test data
    -> find quan/inf wrt S
    -> take avg over all subjects
    """
    df_use = df.copy()
    df_sgrid = df.copy()


    # S grid from empirical test data
    if s_type == "disc": # real/simulation with disc S
        S_names = list( set(XS_names) - set(X_names) )
        if len(S_names) == 1:
            s_min = int(np.min(df_use["S"]))
            s_max = int(np.max(df_use["S"]))
            s_grid = range(s_min, s_max+1)
            if y_fn is None:
                s_grid = [s_grid]
        else:
            s_lst = []
            for s_var in S_names:
                s_min = int(np.min(df_use[s_var]))
                s_max = int(np.max(df_use[s_var]))
                s_grid = range(s_min, s_max+1)
                s_lst.append(np.array(s_grid))
            s_grid = s_lst.copy()
    elif s_type == "cont" and y_fn is not None: # simulation with cont S
        s_grid = df["S"].values.copy()
        assert len(s_grid) > 2
    else:
        assert(1 == 0)


    # S grid from empirical test data
    a_grid = range(0, 2)
    if s_type == "disc":
        s_grid = range(0, 2)
        assert len(s_grid) == 2
    elif s_type == "cont":

        s_grid = df_sgrid["S"].sort_values().values.copy()

        assert len(s_grid) > 2



    # augment X&A with S grid
    df_XA = df_use[[minor_name]+X_names].copy() #contains X
    df_XA["A"] = np.where(df_use[A_name]=="yes", 1,0) #contains X & A
    idx = range(df_XA.shape[0])
    df_XA["idx"] = idx

    if y_fn is None:
        idx_s_lst = [np.array(idx)] + s_grid
        idx_s = np.array(np.meshgrid(*np.array(idx_s_lst, dtype=object))).reshape(1+len(S_names), -1).T
        idx_s = pd.DataFrame(idx_s, columns = ['idx']+S_names)
    else: # if remove, may occur error in .eval(y_fn) # this works as long as there is no multi-S in simulation
        idx_s = np.array(np.meshgrid(idx, s_grid)).reshape(2, len(idx)*len(s_grid)).T
        idx_s = pd.DataFrame(idx_s, columns = ['idx','S'])

    aug_df = df_XA.merge(idx_s, on='idx', how='left').reset_index(drop=True).copy()

    # get E[Y|X,S,A=d(x)]
    if y_fn is not None: # from true distn (simulation)
        aug_df["Y"] = aug_df.eval(y_fn)
    else: # from fitted model (real data)
        Y_pred0 = model_sxa0.predict(aug_df[XS_names]).flatten()
        Y_pred1 = model_sxa1.predict(aug_df[XS_names]).flatten()
        aug_df["Y"] = np.where(aug_df["A"]==0, Y_pred0, Y_pred1)

    # find quantile/inf wrt S
    def quan(x):
        return x.quantile(qua_use)

    minor_df = aug_df[aug_df[minor_name]==1].reset_index(drop=True).copy()
    rest_df = aug_df[aug_df[minor_name]==0].reset_index(drop=True).copy()

    # Gs{E[Y|X,S,A=d(x)]}
    if (s_type == "cont"):
        agg_fn = {"Y": quan}
    elif (s_type == "disc"):
        agg_fn = {"Y": "min"}

    smry_all = aug_df.groupby(by="idx", as_index=False).agg(agg_fn)
    smry_minor = minor_df.groupby(by="idx", as_index=False).agg(agg_fn)
    smry_rest = rest_df.groupby(by="idx", as_index=False).agg(agg_fn)

    # obj: E{G(.)}
    obj_all = np.mean(smry_all.loc[:,"Y"].values)

    if not minor_df.empty:
        obj_minor = np.mean(smry_minor.loc[:,"Y"].values)
    else:
        obj_minor = np.nan

    if not rest_df.empty:
        obj_rest = np.mean(smry_rest.loc[:,"Y"].values)
    else:
        obj_rest = np.nan

    print(A_name, "obj     ",
          "all:",round(obj_all,3),
          "minor:",round(obj_minor,3),
          "rest:",round(obj_rest,3))

    return obj_all, obj_minor, obj_rest



dict mean & std & prop_treat

In [ ]:

def dict_mean(dict_list):
    mean_list = list()
    mean_dict = dict()
    for A_name in dict_list[0].keys():
        mean_dict[A_name] = dict()
        for metric in dict_list[0][A_name].keys():
            mean_dict[A_name][metric] = np.nansum([d[A_name][metric] for d in dict_list]) / len(dict_list)
            mean_list.append(mean_dict[A_name][metric])

    mean_list = np.array(mean_list)
    mean_list = np.reshape(mean_list, (len(dict_list[0].keys()), -1)).T
    mean_df = pd.DataFrame(data=mean_list, columns=dict_list[0].keys())
    mean_df.index = list(dict_list[0][list(dict_list[0].keys())[0]].keys())

    return mean_dict, mean_df


def dict_std(dict_list):
    """
    Replicate this sampling test data procedure 100 times,
    estimate values of each ITR for each replicate,
    and get the averaged estimated values (standard error)
    """
    mean_list = list()
    mean_dict = dict()
    for A_name in dict_list[0].keys():
        mean_dict[A_name] = dict()
        for metric in dict_list[0][A_name].keys():
            mean = np.nansum([d[A_name][metric] for d in dict_list]) / len(dict_list)
            variance = np.nansum([(d[A_name][metric] - mean)**2 for d in dict_list]) / len(dict_list)
            mean_dict[A_name][metric] = variance**0.5 / (len(dict_list))**0.5  #SE
            mean_list.append(mean_dict[A_name][metric])

    mean_list = np.array(mean_list)
    mean_list = np.reshape(mean_list, (len(dict_list[0].keys()), -1)).T
    mean_df = pd.DataFrame(data=mean_list, columns=dict_list[0].keys())
    mean_df.index = list(dict_list[0][list(dict_list[0].keys())[0]].keys())

    return mean_dict, mean_df


def prop_treat(df, A_name, minor_name):
    df_use = df.reset_index(drop=True).copy()

    minor_df = df_use[df_use[minor_name]==1].reset_index(drop=True).copy()
    rest_df = df_use[df_use[minor_name]==0].reset_index(drop=True).copy()

    prop_all = df_use[df_use[A_name]=="yes"].shape[0] / df_use.shape[0]

    if not minor_df.empty:
        prop_minor = minor_df[minor_df[A_name]=="yes"].shape[0] / minor_df.shape[0]
    else:
        prop_minor = np.nan

    if not rest_df.empty:
        prop_rest = rest_df[rest_df[A_name]=="yes"].shape[0] / rest_df.shape[0]
    else:
        prop_rest = np.nan

    print(A_name, "pr_treat",
          "all:",round(prop_all,3),
          "minor:",round(prop_minor,3),
          "rest:",round(prop_rest,3))

    return prop_all, prop_minor, prop_rest


Evaluation Metrics

In [ ]:

def eval_metrics(minor_name, res_df, A_names, X_names, XS_names, y_fn, s_type, is_class,
                    qua_use, fit_Y_XS_A0, fit_Y_XS_A1, fit_qua0, fit_qua1, res_df2):
    print(minor_name)
    print("="*30)
    res = dict()
    for A_name in A_names:
        res[A_name] = dict()
        # metrics
        prop_all, prop_minor, prop_rest = prop_treat(res_df, A_name, minor_name)
        reward_all, reward_minor, reward_rest = expect_reward(res_df, A_name, minor_name)
        value_all, value_minor, value_rest, minorPct = get_value_fn(res_df, A_name, XS_names, minor_name, y_fn, fit_Y_XS_A0, fit_Y_XS_A1)

        if s_type == "disc" or y_fn is not None: # real with disc S | sim with disc/cont S
            obj_all, obj_minor, obj_rest = get_obj_inf(res_df, X_names, XS_names, A_name, minor_name, y_fn, s_type, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1)

        elif s_type == "cont": # real with cont S
            obj_all, obj_minor, obj_rest = get_obj_inf(res_df, X_names, s_type, A_name, minor_name, y_fn, s_type, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1)

        else:
            assert(1 == 0)

        minor_df_train = res_df2[res_df2[minor_name]==1].reset_index(drop=True).copy()
        if not minor_df_train.empty:
            minorPct_train=minor_df_train.shape[0]/res_df2.shape[0]
        else:
            minorPct_train=np.nan

        # save
        res[A_name]["prop_all"] = prop_all;     res[A_name]["prop_minor"] = prop_minor;     res[A_name]["prop_rest"] = prop_rest
        res[A_name]["reward_all"] = reward_all; res[A_name]["reward_minor"] = reward_minor; res[A_name]["reward_rest"] = reward_rest
        res[A_name]["value_all"] = value_all;   res[A_name]["value_minor"] = value_minor;   res[A_name]["value_rest"] = value_rest
        res[A_name]["minorPct_test"] = minorPct; res[A_name]["minorPct_train"] = minorPct_train
        res[A_name]["obj_all"] = obj_all;       res[A_name]["obj_minor"] = obj_minor;       res[A_name]["obj_rest"] = obj_rest
        print("="*30)
    print("="*50)
    return res



In [ ]:
def gen_sim_data_RWDRCT(df,n):

    Spprob=np.exp(-5+3*df['X1']-2*df['S'])/(1+np.exp(-5+3*df['X1']-2*df['S']))
    Sp=np.random.binomial(1,Spprob,n)
    df["Spprob"]=Spprob
    df["Sp"]=Sp
    return df

N=20000
df,_ = gen_sim_data(N,"cont","complex")
df= gen_sim_data_RWDRCT(df,N)
df_RCT=df[df["Sp"]==1]
df_RWD=df[np.random.binomial(1,0.02,df.shape[0])==1]


def transfer_weight_obj(alpha):
    m=df_RWD.shape[0]
    df_RCT_1X=np.concatenate((np.ones((df_RCT.shape[0],1)),df_RCT[["X1","X2","S"]]),axis=1)
    df_RWD_1X=np.concatenate((np.ones((df_RWD.shape[0],1)),df_RWD[["X1","X2","S"]]),axis=1)
    return -(1/N)*np.sum(df_RCT_1X@alpha)+(1/m)*np.sum(np.log(1+np.exp(df_RWD_1X@alpha)))



New Sim Loop

In [ ]:

def gen_sim_data_RWDsplit(df,n):
    probMCAR=np.exp(0.5-1*df['X1']+0*df['X2'])/(1+np.exp(0.5-1*df['X1']+0*df['X2']))
    pMCAR=np.random.binomial(1,probMCAR,n)
    df["pMCAR"]=pMCAR
    return df


In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression

from collections import Counter


In [ ]:
import cvxpy as cp
import numpy as np

def entr(w):
    # Define the entropy function
    return cp.sum(cp.entr(w))

def entr_balance(tol, x_in_rct, x_in_rwe):
    # the optimization problem for the nonparametric weighting method
    n = x_in_rct.shape[0]
    w = cp.Variable(n)
    p = x_in_rct.shape[1]-1
    objective = cp.Maximize(cp.sum(cp.entr(w)))
    constraints = [w >= 0, cp.sum(w) == 1]
    for k in range(p):
        constraints += [cp.abs(cp.sum(w*x_in_rct[:,k+1]) - np.mean(x_in_rwe[:,k+1])) <= tol*np.std(x_in_rwe[:,k+1])*np.sqrt(n/(n-1))]
        constraints += [cp.abs(cp.sum(w*(x_in_rct[:,k+1]**2)) - np.mean(x_in_rwe[:,k+1]**2)) <= tol*np.std(x_in_rwe[:,k+1]**2)*np.sqrt(n/(n-1))]

    prob = cp.Problem(objective, constraints)
    w_tilde = np.zeros(n)
    try:
        result = prob.solve(solver=cp.ECOS)
        w_tilde = w.value
        if np.isnan(w_tilde[0]):
            w_tilde = np.zeros(n)
    except cp.error.SolverError as e:
        # print("error")
        pass
    return w_tilde

def entr_balance_wmid(tol, x_in_rct, x_in_rwe,wmid):
    # the optimization problem for the nonparametric weighting method
    n = x_in_rct.shape[0]
    wmid0 =  x_in_rwe.shape[0] * wmid
    w = cp.Variable(n)
    p = x_in_rct.shape[1]-1
    objective = cp.Maximize(cp.sum(cp.entr(w)))
    constraints = [w >= 0, cp.sum(w) == 1]
    for k in range(p):
        constraints += [cp.abs(cp.sum(w*x_in_rct[:,k+1]) - np.mean(wmid0*x_in_rwe[:,k+1])) <= tol*np.std(wmid0*x_in_rwe[:,k+1])*np.sqrt(n/(n-1))]
        constraints += [cp.abs(cp.sum(w*(x_in_rct[:,k+1]**2)) - np.mean(wmid0*(x_in_rwe[:,k+1]**2))) <= tol*np.std(wmid0*(x_in_rwe[:,k+1]**2))*np.sqrt(n/(n-1))]


    prob = cp.Problem(objective, constraints)
    w_tilde = np.zeros(n)
    try:
        result = prob.solve(solver=cp.ECOS)
        w_tilde = w.value
        if np.isnan(w_tilde[0]):
            w_tilde = np.zeros(n)
    except cp.error.SolverError as e:
        pass
    return w_tilde


def weight_tuned_entr(tols, x_in_rct, x_in_rwe, prop=0.5, smps=10):
    # Evaluate covariate balance with bootstrap samples to choose tuning parameters:
    # the degree of approximate balance sigma_k
    n = x_in_rct.shape[0]
    sds = np.apply_along_axis(np.std, 0, x_in_rwe)
    means = np.apply_along_axis(np.mean, 0, x_in_rwe)
    sds2 = np.apply_along_axis(np.std, 0, x_in_rwe**2)
    moment2 = np.apply_along_axis(np.mean, 0, x_in_rwe**2)
    cov_diff_bars = []
    w_tildes = np.full((n, len(tols)), np.nan)
    for i in range(len(tols)):
        tol = tols[i]
        w_tilde = entr_balance(tol, x_in_rct, x_in_rwe)
        w_tilde[w_tilde < 0] = 0  # the optimization results sometimes return negative weights, so clip them to 0
        if np.sum(w_tilde) == 0:
            cov_diff_bar = 1e+10
        else:
            w_tildes[:, i] = w_tilde
            cov_diffs = []
            for s in range(smps):
                boot_ind = np.random.choice(n, size=int(round(prop * n)), replace=True)
                x_in_rct_boot = x_in_rct[boot_ind, :]
                w_tilde_boot = w_tilde[boot_ind]
                cov_diff = (np.sum(x_in_rct_boot * w_tilde_boot.reshape((-1, 1)), axis=0) / np.sum(w_tilde_boot) - means)[1:] / sds[1:]
                cov_diff_moment2 = (np.sum(x_in_rct_boot**2 * w_tilde_boot.reshape((-1, 1)), axis=0) / np.sum(w_tilde_boot) - moment2)[1:] / sds2[1:]

                cov_diffs.append(np.sum(np.concatenate((cov_diff, cov_diff_moment2))**2))  # L2 measure
            cov_diff_bar = np.mean(cov_diffs)
        cov_diff_bars.append(cov_diff_bar)
    tuned_tol = tols[np.argmin(cov_diff_bars)]
    if min(cov_diff_bars) < 1e+10:
        return w_tildes[:, np.argmin(cov_diff_bars)]
    else:
        raise ValueError('************ optimization not feasible *****************')

def weight_tuned_entr_wmid(tols, x_in_rct, x_in_rwe, wmid, prop=0.5, smps=10):
    # Evaluate covariate balance with bootstrap samples to choose tuning parameters:
    # the degree of approximate balance sigma_k
    n = x_in_rct.shape[0]
    wmid0 =  x_in_rwe.shape[0] * wmid
    sds = np.apply_along_axis(np.std, 0, wmid0[:,np.newaxis]*x_in_rwe)

    means = np.apply_along_axis(np.mean, 0, wmid0[:,np.newaxis]*x_in_rwe)
    sds2 = np.apply_along_axis(np.std, 0, wmid0[:,np.newaxis]*(x_in_rwe**2))

    moment2 = np.apply_along_axis(np.mean, 0, wmid0[:,np.newaxis]*(x_in_rwe**2))
    cov_diff_bars = []
    w_tildes = np.full((n, len(tols)), np.nan)
    for i in range(len(tols)):
        tol = tols[i]
        w_tilde = entr_balance_wmid(tol, x_in_rct, x_in_rwe,wmid)

        w_tilde[w_tilde < 0] = 0  # the optimization results sometimes return negative weights, so clip them to 0
        if np.sum(w_tilde) == 0:
            cov_diff_bar = 1e+10
        else:
            w_tildes[:, i] = w_tilde
            cov_diffs = []
            for s in range(smps):

                boot_ind = np.random.choice(n, size=int(round(prop * n)), replace=True)
                x_in_rct_boot = x_in_rct[boot_ind, :]
                w_tilde_boot = w_tilde[boot_ind]
                cov_diff = (np.sum(x_in_rct_boot * w_tilde_boot.reshape((-1, 1)), axis=0) / np.sum(w_tilde_boot) - means)[1:] / sds[1:]
                cov_diff_moment2 = (np.sum(x_in_rct_boot**2 * w_tilde_boot.reshape((-1, 1)), axis=0) / np.sum(w_tilde_boot) - moment2)[1:] / sds2[1:]

                cov_diffs.append(np.sum(np.concatenate((cov_diff, cov_diff_moment2))**2))  # L2 measure
            cov_diff_bar = np.mean(cov_diffs)
        cov_diff_bars.append(cov_diff_bar)
    tuned_tol = tols[np.argmin(cov_diff_bars)]
    if min(cov_diff_bars) < 1e+10:
        return w_tildes[:, np.argmin(cov_diff_bars)]
    else:
        raise ValueError('************ optimization not feasible *****************')


def learn_weights(y_in_rct, x_in_rct, a_in_rct, x_in_rwe, w_method, misspecify, N):
    # w_method: 1, 2, 3, 4
    n = x_in_rct.shape[0]
    p = x_in_rct.shape[1] - 1
    if w_method == 4:
        # nonparametric weighting
        w_tilde = weight_tuned_entr([0.000, 0.001, 0.002, 0.005, 0.010, 0.020, 0.050, 0.100, 0.200, 0.500, 1.000], x_in_rct, x_in_rwe)
    else:
        raise ValueError("Wrong input for weighting method")
    return w_tilde
def learn_weights_wmid(y_in_rct, x_in_rct, a_in_rct, x_in_rwe, w_method, misspecify, N,wmid):
    # w_method: 1, 2, 3, 4
    n = x_in_rct.shape[0]
    p = x_in_rct.shape[1] - 1
    if w_method == 4:
        # nonparametric weighting
        w_tilde = weight_tuned_entr_wmid([0.000, 0.001, 0.002, 0.005, 0.010, 0.020, 0.050, 0.100, 0.200, 0.500, 1.000], x_in_rct, x_in_rwe,wmid)
    else:
        raise ValueError("Wrong input for weighting method")
    return w_tilde



In [ ]:

def gen_sim_data_RWDRCT(df,n):
    Spprob=np.exp(-2+0*df['X1']-4*df['S'])/(1+np.exp(-2+0*df['X1']-4*df['S']))
    Sp=np.random.binomial(1,Spprob,n)
    df["Spprob"]=Spprob
    df["Sp"]=Sp
    return df


def sim_loop(proj, i_sim, df_all, s_type, X_names, XS_names, names_to_norm, qua_use,
             is_sim, y_fn, is_class, is_qua, is_inf, is_rct,
             is_tune, param_grid, mypath, is_save, is_ones):

    print("*"*15,proj,"i_sim =",i_sim,"begin","*"*15)
    seed = i_sim + 12345
    set_random(seed)

    ########################################################################################################################
    ###################################################### Preprocess ######################################################
    ########################################################################################################################
    df = df_all
    df = gen_sim_data_RWDRCT(df,df.shape[0])


    df_RCT=df[df["Sp"]==1]
    np.random.seed(0)


    np.random.seed(10000)


    df_RWD_0=df[np.random.binomial(1,0.4,df.shape[0])==1]



    df_RWD_tosplit = gen_sim_data_RWDsplit(df_RWD_0,df_RWD_0.shape[0])

    df_RWD_wtcalc = df_RWD_tosplit

    df_RWD = df_RWD_tosplit[df_RWD_tosplit["pMCAR"]==1]
    print("*******NR1",np.sum(df_RWD_tosplit["pMCAR"]==1))
    print("*******NR0",np.sum(df_RWD_tosplit["pMCAR"]==0))

    df_pop=df_RWD.copy()
    wt_type="all"
    if is_ones=="complete":
        wt_type="complete"
        df_RWD_wtcalc = df_RWD_tosplit[df_RWD_tosplit["pMCAR"]==0]
    if is_ones=="impute" or is_ones=="np_impute_old":
        wt_type="impute"
        df_RWD_tosplit_impute = df_RWD_tosplit.copy()
        df_RWD_tosplit_impute["S"][df_RWD_tosplit["pMCAR"]==1]=np.nan

        rf_classifier = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_classifier.fit(df_RWD_tosplit_impute[X_names][df_RWD_tosplit["pMCAR"]==0], df_RWD_tosplit_impute["S"][df_RWD_tosplit["pMCAR"]==0])
        df_RWD_tosplit_impute["S"][df_RWD_tosplit["pMCAR"]==1] = rf_classifier.predict(df_RWD_tosplit_impute[X_names][df_RWD_tosplit["pMCAR"]==1])

        impute_accuracy = mean_squared_error(df_RWD_tosplit["S"][df_RWD_tosplit["pMCAR"]==1], df_RWD_tosplit_impute["S"][df_RWD_tosplit["pMCAR"]==1])

        print(f'ImputeAccuracy: {impute_accuracy}')
        print("*****")

        df_RWD_wtcalc = df_RWD_tosplit_impute.copy()
    if is_ones=="complete" or is_ones=="impute":
        df_RWD_forweights=pd.concat([df_RWD_wtcalc.copy(),df_RCT.copy()]).reset_index(drop=True).copy()
        print("df_RWD_forweights number of rows",df_RWD_forweights.shape[0])


        def transfer_weight_obj(alpha):
            m=df_RWD_forweights.shape[0]
            n=df_RCT.shape[0]


            df_RCT_1X=np.concatenate((np.ones((df_RCT.shape[0],1)),df_RCT[XS_names]),axis=1)
            df_RWD_1X=np.concatenate((np.ones((df_RWD_forweights.shape[0],1)),df_RWD_forweights[XS_names]),axis=1)

            return -(1/m+n)*np.sum(df_RCT_1X@alpha)+(1/m+n)*np.sum(np.log(1+np.exp(df_RWD_1X@alpha)))



        alpha_sol=scipy.optimize.minimize(transfer_weight_obj, x0=np.asarray([0,0,0]), method='Nelder-Mead', tol=1e-6)
        alpha_sol=alpha_sol.x

        df_RCT_1X=np.concatenate((np.ones((df_RCT.shape[0],1)),df_RCT[XS_names]),axis=1)
        transfer_weights=np.exp(-df_RCT_1X@alpha_sol)-1


        alpha_true=np.array([-2,-4])
        print(df_RCT.shape[0],df_RWD.shape[0])
        print(alpha_true)
        print(np.around(alpha_sol,4))

    if is_ones=="impute_only_no":
        wt_type="impute_only"
        df_RWD_tosplit_impute = df_RWD_tosplit.copy()
        df_RWD_tosplit_impute["S"][df_RWD_tosplit["pMCAR"]==1]=np.nan

        rf_classifier = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_classifier.fit(df_RWD_tosplit_impute[X_names][df_RWD_tosplit["pMCAR"]==0], df_RWD_tosplit_impute["S"][df_RWD_tosplit["pMCAR"]==0])
        df_RWD_tosplit_impute["S"][df_RWD_tosplit["pMCAR"]==1] = rf_classifier.predict(df_RWD_tosplit_impute[X_names][df_RWD_tosplit["pMCAR"]==1])

        impute_accuracy = mean_squared_error(df_RWD_tosplit["S"][df_RWD_tosplit["pMCAR"]==1], df_RWD_tosplit_impute["S"][df_RWD_tosplit["pMCAR"]==1])

        print(f'ImputeAccuracy: {impute_accuracy}')
        print("*****")


        df_RWD_wtcalc = df_RWD_tosplit_impute[df_RWD_tosplit["pMCAR"]==1].copy()
    if is_ones=="impute":
        df_RWD["impute_M"]=df_RWD_tosplit_impute["S"][df_RWD_tosplit["pMCAR"]==1]

    if is_ones=="impute_only":
        wt_type="impute_only"
        df_RWD_impar = df_RWD_tosplit[XS_names].copy()  # Select the XS_names columns from RWD
        df_RWD_impar['Source'] = 0  # Add 'Source' column indicating RWD data
        df_RCT_impar = df_RCT[XS_names].copy()  # Select the XS_names columns from RCT
        df_RCT_impar['Source'] = 1  # Add 'Source' column indicating RCT data
        df_impar = pd.concat([df_RWD_impar, df_RCT_impar], axis=0).reset_index(drop=True)
        rf_classifier1 = RandomForestClassifier(random_state=42)
        rf_classifier1.fit(df_impar[X_names], df_impar['Source'])
        prob_RCT1 = np.clip(rf_classifier1.predict_proba(df_RCT[X_names]), 1e-5, 1-1e-5)
        log_regressor1 = LogisticRegression(random_state=42)
        log_regressor1.fit(df_impar[X_names], df_impar['Source'])
        prob_RCT1 = np.clip(log_regressor1.predict_proba(df_RCT[X_names]), 1e-5, 1-1e-5)

        df_RWD_impar_com = df_RWD_tosplit.loc[df_RWD_tosplit["pMCAR"] == 1, XS_names].copy()
        df_RWD_impar_com['Source'] = 0  # Add 'Source' column indicating RWD data
        df_impar_com = pd.concat([df_RWD_impar_com, df_RCT_impar], axis=0).reset_index(drop=True)
        rf_classifier2 = RandomForestClassifier(random_state=42)
        rf_classifier2.fit(df_impar_com[X_names], df_impar_com['Source'])
        prob_RCT2 = np.clip(rf_classifier2.predict_proba(df_RCT[X_names]), 1e-5, 1-1e-5)
        log_regressor2 = LogisticRegression(random_state=42)
        log_regressor2.fit(df_impar_com[X_names], df_impar_com['Source'])
        prob_RCT2 = np.clip(log_regressor2.predict_proba(df_RCT[X_names]), 1e-5, 1-1e-5)

        rf_classifier3 = RandomForestClassifier(random_state=42)
        rf_classifier3.fit(df_impar_com[XS_names], df_impar_com['Source'])
        prob_RCT3 = np.clip(rf_classifier3.predict_proba(df_RCT[XS_names]), 1e-5, 1-1e-5)
        log_regressor3 = LogisticRegression(random_state=42)
        log_regressor3.fit(df_impar_com[XS_names], df_impar_com['Source'])
        prob_RCT3 = np.clip(log_regressor3.predict_proba(df_RCT[XS_names]), 1e-5, 1-1e-5)




        transfer_weights = (prob_RCT1[:, 0]*prob_RCT2[:, 1]*prob_RCT3[:, 0])/(prob_RCT1[:, 1]*prob_RCT2[:, 0]*prob_RCT3[:, 1])

    if is_ones=="np":
        df_RWD_wtcalc = df_RWD_tosplit[df_RWD_tosplit["pMCAR"]==0]
        df_RWD_forweights_2=df_RWD_wtcalc.copy().reset_index(drop=True)
        df_RWD_1X_2=np.concatenate((np.ones((df_RWD_forweights_2.shape[0],1)),df_RWD_forweights_2[XS_names]),axis=1)
        df_RCT_1X=np.concatenate((np.ones((df_RCT.shape[0],1)),df_RCT[XS_names]),axis=1)
        transfer_weights=learn_weights(y_in_rct=0, x_in_rct=df_RCT_1X, a_in_rct=0, x_in_rwe=df_RWD_1X_2, w_method=4, misspecify=0, N=0)
        wt_type="np"
    if is_ones=="np_impute_old":
        wt_type="np_impute_old"
        df_RWD_forweights_2=df_RWD_wtcalc.copy().reset_index(drop=True)
        df_RWD_1X_2=np.concatenate((np.ones((df_RWD_forweights_2.shape[0],1)),df_RWD_forweights_2[XS_names]),axis=1)
        df_RCT_1X=np.concatenate((np.ones((df_RCT.shape[0],1)),df_RCT[XS_names]),axis=1)
        transfer_weights=learn_weights(y_in_rct=0, x_in_rct=df_RCT_1X, a_in_rct=0, x_in_rwe=df_RWD_1X_2, w_method=4, misspecify=0, N=0)
    if is_ones=="np_impute":

        df_RWD_forweights_2=df_RWD_tosplit[df_RWD_tosplit["pMCAR"]==0].copy().reset_index(drop=True)
        df_RWD_1X_2=np.concatenate((np.ones((df_RWD_forweights_2.shape[0],1)),df_RWD_forweights_2[XS_names]),axis=1)
        df_RWD_1X_2_Xonly=np.concatenate((np.ones((df_RWD_forweights_2.shape[0],1)),df_RWD_forweights_2[X_names]),axis=1)
        df_RWD_forweights_2_all=df_RWD_tosplit.copy().reset_index(drop=True)
        df_RWD_1X_2_all=np.concatenate((np.ones((df_RWD_forweights_2_all.shape[0],1)),df_RWD_forweights_2_all[X_names]),axis=1)
        transfer_weights_wmid=learn_weights(y_in_rct=0, x_in_rct=df_RWD_1X_2_Xonly, a_in_rct=0, x_in_rwe=df_RWD_1X_2_all, w_method=4, misspecify=0, N=0)
        df_RCT_1X=np.concatenate((np.ones((df_RCT.shape[0],1)),df_RCT[XS_names]),axis=1)
        transfer_weights=learn_weights_wmid(y_in_rct=0, x_in_rct=df_RCT_1X, a_in_rct=0, x_in_rwe=df_RWD_1X_2, w_method=4, misspecify=0, N=0,wmid=transfer_weights_wmid)
        wt_type="np_impute"


    if is_ones=="ones":
        transfer_weights=np.ones(df_RCT.shape[0])
        wt_type="ones"

    print(np.around(1/transfer_weights[0:10],4))
    print(np.around(np.asarray(df_RCT["Spprob"][0:10]),4))

    set_random(seed)

    df_RCT["TW"]=transfer_weights
    print(np.around(np.asarray(df_RCT["TW"][0:10]),4))
    df_tr_ori = df_RCT.reset_index(drop=True).copy()
    df_te_ori = df_RWD.reset_index(drop=True).copy()
    transfer_weights=df_tr_ori["TW"]


    print("df_tr_ori.head(3)\n", df_tr_ori.head(3))


    df_RCT_TW_st=df_RCT["TW"]/np.sum(df_RCT["TW"])
    print("df_RCT_TW_st",df_RCT_TW_st[:10])
    true_IPSW_st=(1/df_RCT["Spprob"])/np.sum(1/df_RCT["Spprob"])


    cor_wt=np.corrcoef(df_RCT_TW_st,true_IPSW_st)[0,1]
    if s_type == "disc":
        cor_wt=mean_squared_error(df_RCT_TW_st,true_IPSW_st)


    print("*********cor_wt:",cor_wt)

    if i_sim<=10:

        plt.scatter(true_IPSW_st, df_RCT_TW_st)
        plt.xlabel("true IPSW")
        plt.ylabel("estimated_weight_"+wt_type)
        plt.show()


    # normalize
    if not is_sim: # only normalize for real (sim already in [0,1])
        df, df_test = normalize(df_tr_ori, df_te_ori, names_to_norm)
    else:
        df      = df_tr_ori.copy()
        df_test = df_te_ori.copy()
    print("df.head(3)\n", df.head(3))

    # PS based on train
    if is_rct:
        pr_a1 = sum(df["A"]==1) / df.shape[0]
        fit_ps = None
        df_test["ps"] = pr_a1
        df["ps"] = pr_a1
        print("pr_a1", pr_a1)
    else:
        pr_a1 = None

        fit_ps = get_ps_fit(df, XS_names, transfer_weights=df["TW"]) # propensity score A|XS
        df_test["ps"] = fit_ps.predict_proba(df_test[XS_names])[:,1]
        df["ps"] = fit_ps.predict_proba(df[XS_names])[:,1]

    df_test["ps"] = np.where(df_test["A"]==1, df_test["ps"], 1-df_test["ps"])
    df["ps"] = np.where(df["A"]==1, df["ps"], 1-df["ps"])

    # PS trim using percentile cutpoints (only for observational study)
    # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3069059/
    if not is_rct:
        print("for observational study, trimming PS using percentile cutpoints")

        ps_q90 = df["ps"].quantile(0.9)
        ps_q10 = df["ps"].quantile(0.1)
        print("train: before ps min:", np.min(df["ps"]), "ps max:", np.max(df["ps"]))
        df["ps"] = np.where(df["ps"] > ps_q90, ps_q90, df["ps"])
        df["ps"] = np.where(df["ps"] < ps_q10, ps_q10, df["ps"])
        print("train: after  ps min:", np.min(df["ps"]), "ps max:", np.max(df["ps"]))

        ps_q90 = df_test["ps"].quantile(0.9)
        ps_q10 = df_test["ps"].quantile(0.1)
        print("test: before ps min:", np.min(df_test["ps"]), "ps max:", np.max(df_test["ps"]))
        df_test["ps"] = np.where(df_test["ps"] > ps_q90, ps_q90, df_test["ps"])
        df_test["ps"] = np.where(df_test["ps"] < ps_q10, ps_q10, df_test["ps"])
        print("test: after  ps min:", np.min(df_test["ps"]), "ps max:", np.max(df_test["ps"]))

    ########################################################################################################################
    #################################################### Begin Analysis ####################################################
    ########################################################################################################################

    print("begin analysis")

    df0 = df[df["A"]==0].reset_index(drop=True).copy()
    df1 = df[df["A"]==1].reset_index(drop=True).copy()

    df0_te = df_test[df_test["A"]==0].reset_index(drop=True).copy()
    df1_te = df_test[df_test["A"]==1].reset_index(drop=True).copy()

    # Baseline E(Y|X,A) (no S)
    is_base = True
    obj_name = "_base_"
    g_base0, _ = fit_expectation(obj_name, df0, X_names, df, is_class, is_tune, param_grid, df0["TW"], df0_te)
    g_base1, _ = fit_expectation(obj_name, df1, X_names, df, is_class, is_tune, param_grid, df1["TW"], df1_te)

    y_base, w_base = get_label_wt(g_base1, g_base0)
    A_base, A_base_tr, model_base = class_pred(df, df_test, X_names, y_base, w_base, s_type, is_tune, param_grid, df["TW"])

    # EXP E(Y|X,S,A) # XS train, X predict
    is_exp = True
    obj_name = "_exp_"
    g_exp0, _ = fit_expectation(obj_name, df0, XS_names, df, is_class, is_tune, param_grid, df0["TW"], df0_te)
    g_exp1, _ = fit_expectation(obj_name, df1, XS_names, df, is_class, is_tune, param_grid, df1["TW"], df1_te)

    y_exp, w_exp = get_label_wt(g_exp1, g_exp0)
    A_exp, A_exp_tr, model_exp = class_pred(df, df_test, X_names, y_exp, w_exp, s_type, is_tune, param_grid, df["TW"])


    is_MI = True
    obj_name = "_MI_"
    # Assuming A_expOracle_MI is generated in a for loop, we collect all generated arrays
    all_A_expOracle_MI = []  # list to store each A_expOracle_MI

    # Example loop (replace with your actual loop where A_expOracle_MI is generated)
    for reprep in range(n_MI):  # num_loops is the number of iterations


        df_RWD_tosplit_impute_MI = df_RWD_tosplit.copy()
        df_RWD_tosplit_impute_MI["S"][df_RWD_tosplit["pMCAR"]==1]=np.nan

        rf_classifier = RandomForestRegressor(n_estimators=100, random_state=42+reprep*100)
        rf_classifier.fit(df_RWD_tosplit_impute_MI[X_names][df_RWD_tosplit["pMCAR"]==0], df_RWD_tosplit_impute_MI["S"][df_RWD_tosplit["pMCAR"]==0])
        df_RWD_tosplit_impute_MI["S"][df_RWD_tosplit["pMCAR"]==1] = rf_classifier.predict(df_RWD_tosplit_impute_MI[X_names][df_RWD_tosplit["pMCAR"]==1])
        df_RWD_MI = df_RWD_tosplit_impute_MI.copy()
        df_RWD_MI = df_RWD_MI[df_RWD_tosplit["pMCAR"]==1]
        df_test_MI = df_RWD_MI.reset_index(drop=True).copy()

        A_expOracle_MI, _, _ = class_pred(df, df_test_MI, XS_names, y_exp, w_exp, s_type, is_tune, param_grid, df["TW"])
        all_A_expOracle_MI.append(A_expOracle_MI)

    # Convert list of arrays into a 2D NumPy array
    all_A_expOracle_MI = np.array(all_A_expOracle_MI)  # Shape: (num_loops, array_length)

    # Now, find the majority vote for each position
    majority_vote = []

    for col in range(all_A_expOracle_MI.shape[1]):  # Iterate over each column (position)
        votes = all_A_expOracle_MI[:, col]  # Get the votes for the current position
        most_common_vote = Counter(votes).most_common(1)[0][0]  # Get the majority vote
        majority_vote.append(most_common_vote)

    # Convert the result to a NumPy array (if needed)
    final_majority_vote = np.array(majority_vote)
    A_MI = final_majority_vote


    is_expOracle = True
    obj_name = "_expOracle_"

    A_expOracle, A_expOracle_tr, model_expOracle = class_pred(df, df_test, XS_names, y_exp, w_exp, s_type, is_tune, param_grid, df["TW"])

    # QUA/INF E(Y|X,S,A)
    if (is_qua or is_inf):
        # QUA/INF E(Y|X,S,A)
        set_random(seed)
        df0["Yhat"], fit_Y_XS_A0 = fit_expectation("E(Y|X,S,A=0)", df0, XS_names, df0, is_class, is_tune, param_grid, df0["TW"], df0_te)
        df1["Yhat"], fit_Y_XS_A1 = fit_expectation("E(Y|X,S,A=1)", df1, XS_names, df1, is_class, is_tune, param_grid, df1["TW"], df1_te)

        fit_qua0, fit_qua1 = None, None
        if (s_type == "cont"): # quantile: Yhat|X,A (no S)
            set_random(seed)
            fit_qua0, g_qua0 = fit_quantile(df0[X_names], df0["Yhat"], df[X_names], qua_use, is_tune, param_grid, df0["TW"])
            fit_qua1, g_qua1 = fit_quantile(df1[X_names], df1["Yhat"], df[X_names], qua_use, is_tune, param_grid, df1["TW"])

        if (s_type == "disc"): # infinite: try all s and find s that helps achieve a minimal
            g_qua0 = fit_infinite(df, X_names, XS_names, fit_Y_XS_A0)
            g_qua1 = fit_infinite(df, X_names, XS_names, fit_Y_XS_A1)

        y_qua, w_qua = get_label_wt(g_qua1, g_qua0)
        set_random(seed)
        A_qua, A_qua_tr, model_qua = class_pred(df, df_test, X_names, y_qua, w_qua, s_type, is_tune, param_grid, df["TW"])



    ########################################################################################################################
    ################################################### Evaluate Metrics ###################################################
    ########################################################################################################################

    A_names = get_A_names(is_base, is_exp, is_MI, is_expOracle, is_qua, is_inf)
    res_df_te = add_Apred_Midx(df_test, A_names, A_base, A_exp, A_MI, A_expOracle, A_qua)

    A_names.append("A_trueOracle")
    res_df_te_trueOracle=res_df_te.copy()
    res_df_te_trueOracle_A1=res_df_te_trueOracle.copy()
    res_df_te_trueOracle_A1["A1"]="yes"
    res_df_te_trueOracle_A1["Y_A1"]=get_pred_Y(res_df_te_trueOracle_A1, "A1", XS_names, y_fn,None,None)
    res_df_te_trueOracle_A0=res_df_te_trueOracle.copy()
    res_df_te_trueOracle_A0["A0"]="no"
    res_df_te_trueOracle_A0["Y_A0"]=get_pred_Y(res_df_te_trueOracle_A0, "A0", XS_names, y_fn,None,None)
    res_df_te["A_trueOracle"]=np.where(res_df_te_trueOracle_A1["Y_A1"]>=res_df_te_trueOracle_A0["Y_A0"],"yes","no")



    # get optimal A,S given X (QUA/INF optimal)
    minor_name = 'M_opt'


    res_df=df.copy()
    res_df['S_opt'], res_df[minor_name], df_S_opt_train = \
                optimal_AS_inf(df, proj, is_sim, s_type, X_names, XS_names, y_fn, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1, df_pop)

    res_df_te['S_opt'], res_df_te[minor_name], df_S_opt_te = \
                optimal_AS_inf(df_test, proj, is_sim, s_type, X_names, XS_names, y_fn, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1, df_pop)
    if not df_S_opt_te.empty:
        minor_rule(df_S_opt_te, X_names, s_type, "S_opt")

    # metrics
    res_te = eval_metrics(minor_name, res_df_te, A_names, X_names, XS_names, y_fn, s_type, is_class, qua_use, fit_Y_XS_A0, fit_Y_XS_A1, fit_qua0, fit_qua1,res_df)

    # save
    if is_save and i_sim==0:
        # save pred A
        res_df_te.to_csv(mypath+proj+"_"+s_type+"_res_df_te_"+str(i_sim)+".csv", index=False)


    if is_ones=="complete" and i_sim==0:
        res_df.to_csv("drive/MyDrive/data/cont_MAR_df_train_20240128"+str(is_ones)+str(i_sim)+".csv", index=False)

    if is_ones=="impute_only" and i_sim==0:
        res_df_te.to_csv("drive/MyDrive/data/cont_MAR_df_test_20240921"+str(is_ones)+str(i_sim)+".csv", index=False)
        res_df.to_csv("drive/MyDrive/data/cont_MAR_df_train_20240921"+str(is_ones)+str(i_sim)+".csv", index=False)
        df_RWD_tosplit[df_RWD_tosplit["pMCAR"]==0].to_csv("drive/MyDrive/data/cont_MAR_df_rwd2_20240921"+str(is_ones)+str(i_sim)+".csv", index=False)





    print("*"*15,proj,"i_sim=",i_sim,"finish","*"*15)

    return res_te, cor_wt


In [ ]:


def dict_list_to_df(dict_list):
    dim1_temp=len(dict_list[0].keys())
    colname_temp=dict_list[0].keys()
    index_temp=[*dict_list[0][list(dict_list[0].keys())[0]].keys()]

    for idx_dict_cell in range(len(dict_list)):
        print(idx_dict_cell)
        dict_cell=list()
        for A_name in dict_list[0].keys():
            for metric in dict_list[0][A_name].keys():
                dict_cell.append(dict_list[idx_dict_cell][A_name][metric])
        dict_cell = np.array(dict_cell)
        dict_cell = np.reshape(dict_cell, (dim1_temp, -1)).T
        dict_cell_df = pd.DataFrame(data=dict_cell, columns=colname_temp)
        dict_cell_df["metrics"] = index_temp
        dict_cell_df["loop"]=idx_dict_cell
        if idx_dict_cell==0:
            dict_df=dict_cell_df
        else:
            dict_df=pd.concat([dict_df,dict_cell_df])

    return dict_df


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:

# settings
n_MI=5
proj    = "simulation"

y_set   = "toy"
s_type  = "cont"


qua_use = 0.2

n_train = 10000 #train-test: 8-2
n_sim   = 100

n_train = 15000 #train-test: 8-2
n_sim   = 100
is_tune = False
is_save = False
mypath  = ""

if (proj == "simulation"):
    df_all_ori, y_fn = gen_sim_data(n_train, s_type, y_set)
    is_sim = True
else:
    is_sim = False
    y_fn = None
    y_set = None

if proj in ["simulation"]:
    is_class = False

if (s_type == "disc"):
    is_qua = False
    is_inf = True
else: #cont
    is_qua = True
    is_inf = False

if y_set in ['toy','noise']:
    is_rct = True
elif y_set in ['complex','positivity','unconfound']:
    is_rct = False
else:
    assert(1==0)

if (is_tune):
    param_grid = dict(layers=[1,2,3],
                      nodes=[256,512,1024],
                      dropouts=[0.1,0.2,0.3],
                      acts=["sigmoid","relu","tanh"],
                      opts=["adam","nadam","adadelta"],
                      bsizes=[32,64,128],
                      n_epochs=[50,100,200]
                     )
else:
    param_grid = None

if not os.path.exists('logs'):
    os.makedirs('logs')

# names of all X
df_all = df_all_ori.copy()
if is_sim:
    X_names = list(df_all.columns[np.flatnonzero(np.char.startswith(list(df_all.columns), 'X'))])
    S_names = ["S"]
else:
    X_names = list( set(df_all.columns) - set(["A","Y","S"]) )
    S_names = ["S"]


X_names=["X1"]
XS_names = S_names + X_names

if not is_sim:
    names_to_norm = list(df_all[XS_names].nunique()[df_all[XS_names].nunique() > 2].index)
else:
    names_to_norm = []
print("X_names:", X_names)
print("S_names:", S_names)
print("cont vars (need to normalize):", names_to_norm)
print("disc vars (no need to normalize):", list(set(XS_names) - set(names_to_norm)))

# begin running
i_sim = 0
dict_list_est = list()


dict_list_est_ones = list()
dict_list_est_all = list()
dict_list_est_impute = list()
dict_list_est_impute_only = list()

cor_list_est = list()
cor_list_est_ones = list()
cor_list_est_all = list()
cor_list_est_impute = list()
cor_list_est_impute_only = list()


dict_list_est_ones2 = list()
dict_list_est_impute2 = list()
dict_list_est_impute_only2 = list()
cor_list_est_ones2 = list()
cor_list_est_impute2 = list()
cor_list_est_impute_only2 = list()

start0 = time.time()
start_time = time.strftime("%H:%M:%S", time.localtime())

for i_sim in range(50,n_sim,1):

    start = time.time()

    if is_sim:
        set_random(i_sim+12345)
        df_all_ori, y_fn = gen_sim_data(n_train, s_type, y_set)
        df_all = df_all_ori.copy()



    res_est, cor_wt = sim_loop(proj, i_sim, df_all, s_type, X_names, XS_names, names_to_norm, qua_use,
          is_sim, y_fn, is_class, is_qua, is_inf, is_rct,
          is_tune, param_grid, mypath, is_save, is_ones="np")

    dict_list_est.append(res_est)
    cor_list_est.append(cor_wt)

    res_est_ones, cor_wt_ones = sim_loop(proj, i_sim, df_all, s_type, X_names, XS_names, names_to_norm, qua_use,
             is_sim, y_fn, is_class, is_qua, is_inf, is_rct,
             is_tune, param_grid, mypath, is_save, is_ones="complete")

    dict_list_est_ones.append(res_est_ones)
    cor_list_est_ones.append(cor_wt_ones)


    res_est_impute, cor_wt_impute = sim_loop(proj, i_sim, df_all, s_type, X_names, XS_names, names_to_norm, qua_use,
             is_sim, y_fn, is_class, is_qua, is_inf, is_rct,
             is_tune, param_grid, mypath, is_save, is_ones="impute")

    dict_list_est_impute.append(res_est_impute)
    cor_list_est_impute.append(cor_wt_impute)


    res_est_impute_only, cor_wt_impute_only = sim_loop(proj, i_sim, df_all, s_type, X_names, XS_names, names_to_norm, qua_use,
             is_sim, y_fn, is_class, is_qua, is_inf, is_rct,
             is_tune, param_grid, mypath, is_save, is_ones="ones")


    dict_list_est_impute_only.append(res_est_impute_only)
    cor_list_est_impute_only.append(cor_wt_impute_only)

    res_est_impute2, cor_wt_impute2 = sim_loop(proj, i_sim, df_all, s_type, X_names, XS_names, names_to_norm, qua_use,
             is_sim, y_fn, is_class, is_qua, is_inf, is_rct,
             is_tune, param_grid, mypath, is_save, is_ones="np_impute")

    dict_list_est_impute2.append(res_est_impute2)
    cor_list_est_impute2.append(cor_wt_impute2)


    res_est_ones2, cor_wt_ones2 = sim_loop(proj, i_sim, df_all, s_type, X_names, XS_names, names_to_norm, qua_use,
             is_sim, y_fn, is_class, is_qua, is_inf, is_rct,
             is_tune, param_grid, mypath, is_save, is_ones="np_impute_old")

    dict_list_est_ones2.append(res_est_ones2)
    cor_list_est_ones2.append(cor_wt_ones2)

    res_est_impute_only2, cor_wt_impute_only2 = sim_loop(proj, i_sim, df_all, s_type, X_names, XS_names, names_to_norm, qua_use,
             is_sim, y_fn, is_class, is_qua, is_inf, is_rct,
             is_tune, param_grid, mypath, is_save, is_ones="impute_only")


    dict_list_est_impute_only2.append(res_est_impute_only2)
    cor_list_est_impute_only2.append(cor_wt_impute_only2)




    if i_sim % 1 == 0:
        _, tmp_df = dict_mean(dict_list_est)
        _, tmp_std_df = dict_std(dict_list_est)
        _, tmp_df_ones = dict_mean(dict_list_est_ones)
        _, tmp_std_df_ones = dict_std(dict_list_est_ones)

        _, tmp_df_impute = dict_mean(dict_list_est_impute)
        _, tmp_std_df_impute = dict_std(dict_list_est_impute)

        _, tmp_df_impute_only = dict_mean(dict_list_est_impute_only)
        _, tmp_std_df_impute_only = dict_std(dict_list_est_impute_only)

        _, tmp_df_ones2 = dict_mean(dict_list_est_ones2)
        _, tmp_std_df_ones2 = dict_std(dict_list_est_ones2)
        _, tmp_df_impute2 = dict_mean(dict_list_est_impute2)
        _, tmp_std_df_impute2 = dict_std(dict_list_est_impute2)
        _, tmp_df_impute_only2 = dict_mean(dict_list_est_impute_only2)
        _, tmp_std_df_impute_only2 = dict_std(dict_list_est_impute_only2)


        mean_cor_impute=np.mean(cor_list_est_impute)
        mean_cor_impute_only=np.mean(cor_list_est_impute_only)
        mean_cor=np.mean(cor_list_est)
        mean_cor_ones=np.mean(cor_list_est_ones)

        mean_cor_impute2=np.mean(cor_list_est_impute2)
        mean_cor_impute_only2=np.mean(cor_list_est_impute_only2)
        mean_cor_ones2=np.mean(cor_list_est_ones2)
        print(i_sim, "M_test cor_ones\n", mean_cor_ones)
        print(i_sim, "M_test mean_ones\n", tmp_df_ones.round(3))
        print(i_sim, "M_test std_ones\n",  tmp_std_df_ones.round(3))

        print(i_sim, "M_test cor_complete\n", mean_cor)
        print(i_sim, "M_test mean_complete\n", tmp_df.round(3))
        print(i_sim, "M_test std_complete\n",  tmp_std_df.round(3))


        print(i_sim, "M_test cor_impute\n", mean_cor_impute)
        print(i_sim, "M_test mean_impute\n", tmp_df_impute.round(3))
        print(i_sim, "M_test std_impute\n",  tmp_std_df_impute.round(3))


        print(i_sim, "M_test cor_impute_only\n", mean_cor_impute_only)
        print(i_sim, "M_test mean_impute_only\n", tmp_df_impute_only.round(3))
        print(i_sim, "M_test std_impute_only\n",  tmp_std_df_impute_only.round(3))



        if (i_sim+1) % 5 == 0:

            dict_df = dict_list_to_df(dict_list_est)
            dict_df_ones = dict_list_to_df(dict_list_est_ones)
            dict_df_impute = dict_list_to_df(dict_list_est_impute)
            dict_df_impute_only = dict_list_to_df(dict_list_est_impute_only)
            dict_df_ones2 = dict_list_to_df(dict_list_est_ones2)
            dict_df_impute2 = dict_list_to_df(dict_list_est_impute2)
            dict_df_impute_only2 = dict_list_to_df(dict_list_est_impute_only2)
            dict_df_ones.to_csv("drive/MyDrive/data/result_cont_MAR_20241210_complete_par"+str(i_sim)+".csv")
            dict_df_ones2.to_csv("drive/MyDrive/data/result_cont_MAR_20241210_np_impute_old"+str(i_sim)+".csv")
            dict_df.to_csv("drive/MyDrive/data/result_cont_MAR_20241210_complete_np"+str(i_sim)+".csv")
            dict_df_impute.to_csv("drive/MyDrive/data/result_cont_MAR_20241210_old_impute_par"+str(i_sim)+".csv")
            dict_df_impute_only.to_csv("drive/MyDrive/data/result_cont_MAR_20241210_ones"+str(i_sim)+".csv")
            dict_df_impute2.to_csv("drive/MyDrive/data/result_cont_MAR_20241210_impute_np"+str(i_sim)+".csv")
            dict_df_impute_only2.to_csv("drive/MyDrive/data/result_cont_MAR_20241210_impute_par"+str(i_sim)+".csv")

        print("*"*60)

    end = time.time()
    print("iter", i_sim, "takes", (end - start)//60, "mins")
    print("start time", start_time)
    print("current process takes", (end - start0)//60, "mins")
    print("*"*80)






end0 = time.time()
print("whole process takes", (end0 - start0)//60, "mins")
print("*"*80)

